In [1]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import TopSimilarRecommender as TSR
import random

In [3]:
recommendations= pd.read_csv('Submissions/top_similar_submission_0.04744.csv')
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

In [4]:
big_target_playlist = [i for i in np.unique(train['playlist_id'].values) if train[train['playlist_id'] == i].shape[0] >= 10]

In [5]:
test_pl = pd.DataFrame({"playlist_id": list(map(lambda x: random.choice(big_target_playlist), range(100)))})
test_pl

,playlist_id
0,5521188
1,225179
2,160503
3,11319332
4,6571054
5,2586071
6,3223162
7,8485627
8,10943483
9,7665603


In [6]:
tracks_in_test_pl = pd.merge(train, test_pl, how='inner', on='playlist_id')
tracks_removed= pd.DataFrame(columns=['playlist_id', 'track_id'], dtype='int32')
indexes_to_remove= np.array([], dtype='int32')
for i in np.unique(tracks_in_test_pl['playlist_id'].values):
    #randomly selects 5 elements from playlist i and save indexes
    tmp = (tracks_in_test_pl.where(tracks_in_test_pl['playlist_id']==i).dropna().sample(5)).index
    #insert the items selected in tracks_removed dataFrame
    tracks_removed= tracks_removed.append(tracks_in_test_pl.take(tmp)).astype(int)
    #remove tracks from the original dataframe
    indexes_to_remove=np.append(indexes_to_remove, values=tmp)

tracks_in_train= pd.DataFrame(tracks_in_test_pl)
tracks_in_train.drop(indexes_to_remove, inplace='true')
tracks_removed

,playlist_id,track_id
4341,160503,3835624
4329,160503,1344294
4334,160503,1729875
4328,160503,853629
4339,160503,2816088
3155,225179,3872220
3165,225179,3735034
3162,225179,2726028
3159,225179,699515
3170,225179,2553607


In [8]:
map_eval = rs.evaluate(recommendations, tracks_removed, 'MAP')

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().